In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
matchdata=pd.read_csv(r'C:\Users\Yogesh Arora\Downloads\IPL Ball-by-Ball 2008-2020\IPL Ball-by-Ball 2008-2020.csv')

In [3]:
matchdata.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [17]:
        batsmen_score = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
        batsmen_ball_faced = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['over'].count()).\
            rename(columns={"over": "total_balls_faced"})
        batsmen_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'batsman'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_faced"})
        batsmen_scores6 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_6"})
        batsmen_scores4 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_4"})
        batsmen_position = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].min())
        df_list = [batsmen_score, batsmen_ball_faced, batsmen_ball_faced_legal, batsmen_scores6, batsmen_scores4, batsmen_position]
        batsmen_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        batsmen_summary = pd.merge(batsmen_summary, matchdata[['id', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                                   drop_duplicates(), on=['id', 'batsman'], how='left')
        batsmen_summary.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'batsmen_battingteam',
                                        'bowling_team': 'batsmen_bowlingteam'}, inplace=True)
       

In [18]:
batsmen_summary

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,batsmen_innings,batsmen_battingteam,batsmen_bowlingteam
0,335982,AA Noffke,9,12,11.0,NaN,1.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
1,335982,B Akhil,0,2,2.0,NaN,NaN,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
2,335982,BB McCullum,158,77,73.0,13.0,10.0,0,1,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,CL White,6,10,8.0,NaN,NaN,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
4,335982,DJ Hussey,12,12,12.0,NaN,1.0,0,1,Kolkata Knight Riders,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...
12157,1237181,RR Pant,56,40,38.0,2.0,4.0,0,1,Delhi Capitals,Mumbai Indians
12158,1237181,S Dhawan,15,13,13.0,NaN,3.0,0,1,Delhi Capitals,Mumbai Indians
12159,1237181,SA Yadav,19,20,20.0,1.0,1.0,0,2,Mumbai Indians,Delhi Capitals
12160,1237181,SO Hetmyer,5,5,5.0,NaN,1.0,0,1,Delhi Capitals,Mumbai Indians


In [19]:
        bowler_wickets = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "total_wickets"})
        bowler_overs_bow = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].count()).\
            rename(columns={"over": "total_balls_bowled"})
        bowler_ball_faced_legal = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].nunique()).\
            rename(columns={"over": "total_legal_balls_bowled"})
        # calculating the number of maiden overs bowled in a match
        bowler_runs_given = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs_given"})
        #matchdata_runs_per_over = pd.DataFrame(self.matchdata.groupby(['matchid', 'bowlername', 'over_num'])['batsmanscorevalue'].sum()).reset_index()
        #matchdata_runs_per_over['maiden_overs'] = np.where(matchdata_runs_per_over['batsmanscorevalue'] == 0, 1, 0)
        #matchdata_runs_per_over = matchdata_runs_per_over.groupby(['matchid', 'bowlername'])['maiden_overs'].sum()
        # combining all the features created so far
        df_list = [bowler_wickets, bowler_overs_bow, bowler_ball_faced_legal, bowler_runs_given]
        bowler_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        # adding additional columns just in case we need for modeling
        bowler_summary = pd.merge(bowler_summary, matchdata[['id', 'bowler', 'inning', 'batting_team', 'bowling_team']].drop_duplicates(), on=['id', 'bowler'], how='left')
        bowler_summary.rename(columns={'inning': 'bowlers_innings', 'bowler': 'playername', 'batting_team': 'bowler_battingteam',
                                       'bowling_team': 'bowler_bowlingteam'}, inplace=True)
        bowler_summary['economy_rate'] = (bowler_summary['total_runs_given'] * 6) / bowler_summary['total_legal_balls_bowled']

In [20]:
bowler_summary

,id,playername,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,bowler_battingteam,bowler_bowlingteam,economy_rate
0,335982,AA Noffke,1.0,25,4,35,1,Kolkata Knight Riders,Royal Challengers Bangalore,52.5
1,335982,AB Agarkar,3.0,28,4,21,2,Royal Challengers Bangalore,Kolkata Knight Riders,31.5
2,335982,AB Dinda,2.0,20,3,7,2,Royal Challengers Bangalore,Kolkata Knight Riders,14.0
3,335982,CL White,NaN,7,1,22,1,Kolkata Knight Riders,Royal Challengers Bangalore,132.0
4,335982,I Sharma,1.0,19,3,6,2,Royal Challengers Bangalore,Kolkata Knight Riders,12.0
...,...,...,...,...,...,...,...,...,...,...
9662,1237181,MP Stoinis,1.0,12,2,23,2,Mumbai Indians,Delhi Capitals,69.0
9663,1237181,NM Coulter-Nile,2.0,25,4,28,1,Delhi Capitals,Mumbai Indians,42.0
9664,1237181,P Dubey,NaN,18,3,29,2,Mumbai Indians,Delhi Capitals,58.0
9665,1237181,R Ashwin,NaN,24,4,28,2,Mumbai Indians,Delhi Capitals,42.0


In [21]:
  ipl_merged_scorecard = pd.merge(batsmen_summary, bowler_summary, on=['id', 'playername'], how='outer')

In [24]:
        MINAVGBALLSFACED = 8
        MINAVGBOWLSBOWLED = 6
        player_avg = ipl_merged_scorecard[['playername', 'total_balls_faced', 'total_balls_bowled']].fillna(0)
        player_avg = pd.DataFrame(player_avg.groupby('playername')[['total_balls_faced', 'total_balls_bowled']].mean())
        conditions = [((player_avg['total_balls_faced'] >= MINAVGBALLSFACED) & (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)),
                      (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)]
        choices = ['AllRounder', 'Bowler']
        player_avg['playing_role'] = np.select(conditions, choices, default='Batsmen')
        player_avg = player_avg.reset_index()

In [22]:
 ipl_merged_scorecard

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,batsmen_innings,batsmen_battingteam,batsmen_bowlingteam,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,bowler_battingteam,bowler_bowlingteam,economy_rate
0,335982,AA Noffke,9.0,12.0,11.0,NaN,1.0,0.0,2.0,Royal Challengers Bangalore,Kolkata Knight Riders,1.0,25.0,4.0,35.0,1.0,Kolkata Knight Riders,Royal Challengers Bangalore,52.5
1,335982,B Akhil,0.0,2.0,2.0,NaN,NaN,0.0,2.0,Royal Challengers Bangalore,Kolkata Knight Riders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,BB McCullum,158.0,77.0,73.0,13.0,10.0,0.0,1.0,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,CL White,6.0,10.0,8.0,NaN,NaN,0.0,2.0,Royal Challengers Bangalore,Kolkata Knight Riders,NaN,7.0,1.0,22.0,1.0,Kolkata Knight Riders,Royal Challengers Bangalore,132.0
4,335982,DJ Hussey,12.0,12.0,12.0,NaN,1.0,0.0,1.0,Kolkata Knight Riders,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17146,1237181,K Rabada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18.0,3.0,32.0,2.0,Mumbai Indians,Delhi Capitals,64.0
17147,1237181,NM Coulter-Nile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,25.0,4.0,28.0,1.0,Delhi Capitals,Mumbai Indians,42.0
17148,1237181,P Dubey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,3.0,29.0,2.0,Mumbai Indians,Delhi Capitals,58.0
17149,1237181,R Ashwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,4.0,28.0,2.0,Mumbai Indians,Delhi Capitals,42.0


In [25]:
player_avg

,playername,total_balls_faced,total_balls_bowled,playing_role
0,A Ashish Reddy,7.000000,9.642857,Bowler
1,A Chandila,0.583333,19.500000,Bowler
2,A Chopra,12.500000,0.000000,Batsmen
3,A Choudhary,4.000000,21.600000,Bowler
4,A Dananjaya,5.000000,25.000000,Bowler
...,...,...,...,...
575,YV Takawale,18.300000,0.000000,Batsmen
576,Yashpal Singh,16.750000,0.000000,Batsmen
577,Younis Khan,7.000000,0.000000,Batsmen
578,Yuvraj Singh,17.093023,6.837209,AllRounder
